### Diseño de Agentes Inteligentes
### Tarea 2. Agentes Solucionadores de Problemas (PSA) y búsqueda

*Integrantes del Equipo 4* 
1. Ricardo Kaleb Flores Alfonso     A01198716
2. Sebastián Miramontes Soto        A01285296
3. Raúl Correa Ocañas               A01722401


#### Cleanup Puzzle

Cleanup Puzzle es un juego de mesa que requiere que limpies el tablero. Para ello, deberás hacer clic en las fichas de un tablero tipo cuadrícula. Cada ficha puede estar vacía o contener una bola. El mosaico en el que haga clic no se verá afectado, pero sus vecinos directos (horizontal y verticalmente) se invertirán (si el mosaico contiene una bola, se vaciará y viceversa). El objetivo es determinar una secuencia de clics que elimine todas las bolas del tablero.

El juego, que puedes encontrar y jugar en el enlace https://www.mathsisfun.com/games/cleanup-puzzle.html , consiste en un tablero de 11x11 que se juega en una secuencia de etapas de complejidad incrementada. La complejidad de un escenario está relacionada con la cantidad de bolas en el tablero y sus ubicaciones. En el juego en línea, hay un límite de clics definido para cada etapa y el juego termina cuando no puede encontrar una secuencia de clics lo suficientemente pequeña para limpiar el tablero.
Ejemplos de diferentes etapas del juego en línea:

![alt text](ejemplo1.png)

![alt text](ejemplo2.png)


In [14]:
# Simpleai.search es la librería desde la que importaremos los algoritmos de búsqueda
from simpleai.search import SearchProblem, astar, greedy
# BaseViewer nos ayuda a resumir los resultados de la búsqueda
from simpleai.search.viewers import BaseViewer, WebViewer

PSA de Busqueda Informada utilizando el algoritmo A* y Greedy

In [15]:
class Cleanup(SearchProblem):
    # Definimos el problema de búsqueda. Cleanup hereda de SearchProblem y toma como parametros
    # el estado inicial y el estado meta
    def __init__(self, initial, goal):
        super().__init__(initial_state = initial)
        self.goal_state = goal

    def actions(self, state):
        # No consideraremos todas las coordenadas, solo aquellas que
        # modifiquen las bolitas adyacentes.
        valid_actions = []
        rows = len(state)
        cols = len(state[0])

        # Si la bolita está a la izquierda, arriba, abajo o derecha de la celda actual,
        # entonces es una acción válida
        for i in range(rows):
            for j in range(cols):

                # Checa si hay una bolita en la celda de arriba
                if i - 1 >= 0 and state[i - 1][j]:
                    valid_actions.append((i, j))

                # Checa si hay una bolita en la celda de abajo
                if i + 1 < rows and state[i + 1][j]:
                    valid_actions.append((i, j))

                # Checa si hay una bolita en la celda de la izquierda
                if j - 1 >= 0 and state[i][j - 1]:
                    valid_actions.append((i, j))

                # Checa si hay una bolita en la celda de la derecha
                if j + 1 < cols and state[i][j + 1]:
                    valid_actions.append((i, j))
        
        return valid_actions # Regresa la lista de acciones válidas
    
    def result(self, state, action):
        i, j = action # Obtiene las coordenadas de la acción
        # Crea una copia del estado actual
        child_state = [list(row) for row in state]
        
        # Cambia el estado de las bolitas adyacentes
        # Revisa que sea posible poner una bolita en la celda de arriba
        if i - 1 >= 0:
            # Cambia el estado de la bolita de arriba
            child_state[i - 1][j] = not child_state[i - 1][j]

        # Revisa que sea posible poner una bolita en la celda de la izquierda
        if j - 1 >= 0:
            # Cambia el estado de la bolita de la izquierda
            child_state[i][j - 1] = not child_state[i][j - 1]
        
        # Revisa que sea posible poner una bolita en la celda de la derecha
        if j + 1 < len(state[0]):
            # Cambia el estado de la bolita de la derecha
            child_state[i][j + 1] = not child_state[i][j + 1]
        
        # Revisa que sea posible poner una bolita en la celda de abajo
        if i + 1 < len(state):
            # Cambia el estado de la bolita de abajo
            child_state[i + 1][j] = not child_state[i + 1][j]

        return tuple([tuple(row) for row in child_state]) # Regresa el nuevo estado
    
    # Checa si el estado actual es el estado meta
    def is_goal(self, state):
        return state == self.goal_state
    
    # Define el costo de una acción, en este caso, todas las acciones tienen costo 1
    def cost(self, state, action, state2):
        return 1

    # Define la heurística a utilizar
    def heuristic(self, state):
        rows = len(state)
        cols = len(state[0])
        ball_positions = []
        goal_positions = []

        for i in range(rows):
            for j in range(cols):
                if state[i][j]:
                    # Guarda las posiciones de las bolitas
                    ball_positions.append((i, j))
                else:
                    # Guarda las posiciones de las celdas vacías
                    goal_positions.append((i, j))

        # Calcula la distancia mínima de cada bolita a una celda vacía
        heuristic_value = 0
        for ball_pos in ball_positions:
            # También conocido como Manhattan distance
            min_distance = min(abs(ball_pos[0] - goal_pos[0]) + abs(ball_pos[1] - goal_pos[1]) for goal_pos in goal_positions)
            heuristic_value += min_distance

        return heuristic_value

In [16]:
# Función para mostrar el resultado de la búsqueda
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i, action)
                print('¡Meta lograda con costo = ', result.cost,'!')
            else:
                print(i, action)

            print('  ', state)
    else:
        print('Mala configuración del problema')

In [17]:
# Funcion para crear un tablero de tamaño rows x cols vacio como listas de listas
def list_board(rows, cols):
        return [[False for i in range(cols)] for j in range(rows)]

# Funcion para crear un tablero de tamaño rows x cols con n como tuplas de tuplas
def tuple_board(board):
        return tuple([tuple(row) for row in board])

In [18]:
# Tamaño del tablero: 11x11
rows, cols = 11,11
# init = list_board(rows, cols)
# El estado meta es un tablero sin bolitas
goal = list_board(rows, cols)

In [19]:
# # Dificultad 1

# init[3][0] = True
# init[4][1] = True
# init[5][0] = True

# init[10][8] = True
# init[9][9] = True
# init[10][10] = True

In [20]:
# # Dificultad 2
# init[2][4] = True
# init[3][3] = True
# init[3][5] = True
# init[4][4] = True

# init[9][7] = True
# init[7][7] = True
# init[8][6] = True
# init[8][8] = True

# init[7][0] = True
# init[6][1] = True
# init[8][1] = True

# init[10][4] = True
# init[9][5] = True
# init[10][6] = True

# init[7][4] = True
# init[6][3] = True
# init[8][3] = True

In [21]:
# # Dificultad 3
# init = ((False, False, False, True, False, False, False, False, False, False, False),
#         (False, False, True, False, True, False, False, False, False, False, False),
#         (False, False, False, True, False, False, False, False, False, False, False),
#         (False, False, False, False, False, False, False, True, False, False, False),
#         (False, False, False, True, False, False, True, False, False, False, False),
#         (False, False, True, False, True, False, False, False, False, True, False),
#         (False, False, False, True, False, False, False, False, True, True, False),
#         (False, False, False, False, False, False, False, False, True, False, True),
#         (True, False, False, False, False, True, False, False, False, True, False),
#         (False, True, False, False, True, False, True, False, False, False, False),
#         (True, False, False, False, False, True, False, False, False, False, False))

In [22]:
# Dificultad 4
init = ((False, False, False, False, False, False, False, True, False, False, False),
        (False, False, True, False, True, False, False, True, True, False, False),
        (False, False, False, True, False, True, True, True, True, False, False),
        (True, False, False, False, False, False, False, True, False, False, False),
        (False, True, False, False, False, False, False, False, False, False, False),
        (True, False, False, False, False, False, False, False, False, False, True),
        (False, False, False, False, False, False, False, False, False, True, False),
        (False, False, False, False, False, False, False, False, True, True, True),
        (False, False, False, False, True, False, False, True, True, True, True),
        (True, False, False, True, False, True, False, False, True, False, False),
        (False, True, False, False, True, False, False, False, True, False, True))

In [23]:
# Entregar el tablero inicial y el tablero meta como tuplas de tuplas
init = tuple_board(init)
goal = tuple_board(goal)

# Visualizar el tablero inicial
init_int = [[int(cell) for cell in row] for row in init]
init_int

[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
 [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]]

In [24]:
# Busqueda de A* con BaseViewer

astar_viewer = BaseViewer()       # Solo estadísticas
result = astar(Cleanup(init,goal), graph_search=True, viewer=astar_viewer)
print()
print('>> Búsqueda A* <<')
display(result)


>> Búsqueda A* <<
Configuración inicial
   ((False, False, False, False, False, False, False, True, False, False, False), (False, False, True, False, True, False, False, True, True, False, False), (False, False, False, True, False, True, True, True, True, False, False), (True, False, False, False, False, False, False, True, False, False, False), (False, True, False, False, False, False, False, False, False, False, False), (True, False, False, False, False, False, False, False, False, False, True), (False, False, False, False, False, False, False, False, False, True, False), (False, False, False, False, False, False, False, False, True, True, True), (False, False, False, False, True, False, False, True, True, True, True), (True, False, False, True, False, True, False, False, True, False, False), (False, True, False, False, True, False, False, False, True, False, True))
1 (7, 9)
   ((False, False, False, False, False, False, False, True, False, False, False), (False, False, True, False,

In [25]:
# Busqueda de Greedy con BaseViewer

greedy_viewer = BaseViewer()       # Solo estadísticas
result = greedy(Cleanup(init,goal), graph_search=True, viewer=greedy_viewer)
print()
print('>> Greedy <<')
display(result)


>> Greedy <<
Configuración inicial
   ((False, False, False, False, False, False, False, True, False, False, False), (False, False, True, False, True, False, False, True, True, False, False), (False, False, False, True, False, True, True, True, True, False, False), (True, False, False, False, False, False, False, True, False, False, False), (False, True, False, False, False, False, False, False, False, False, False), (True, False, False, False, False, False, False, False, False, False, True), (False, False, False, False, False, False, False, False, False, True, False), (False, False, False, False, False, False, False, False, True, True, True), (False, False, False, False, True, False, False, True, True, True, True), (True, False, False, True, False, True, False, False, True, False, False), (False, True, False, False, True, False, False, False, True, False, True))
1 (7, 9)
   ((False, False, False, False, False, False, False, True, False, False, False), (False, False, True, False, True

In [26]:
# Estadísticas de las búsquedas

if astar_viewer != None:
    print('A* Stats:')
    print(astar_viewer.stats)

print()

if greedy_viewer != None:
    print('Greedy Stats:')
    print(greedy_viewer.stats)

A* Stats:
{'max_fringe_size': 450, 'visited_nodes': 20, 'iterations': 20}

Greedy Stats:
{'max_fringe_size': 420, 'visited_nodes': 17, 'iterations': 17}
